# Scrape India

Get the site : https://www.carwale.com/used/cars-for-sale/

## Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.firefox.options import Options
import pandas as pd 
import numpy as np 

import random
from selenium.webdriver.common.action_chains import ActionChains
from itertools import combinations
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from fake_useragent import UserAgent

## Fonctions Annexe

In [2]:
def click_button(driver, xpath):
    try:
        button = driver.find_element(By.XPATH, xpath)
        button.click()
        return True
    except NoSuchElementException:
        print(f"Le bouton avec l'xpath '{xpath}' est introuvable.")
        return False

In [3]:
def scrape_content(driver, xpath, content_type='text', max_retries=3, timeout=8):
    for retry in range(max_retries):
        try:
            # Trouver tous les éléments correspondant à l'XPath fourni
            elements = WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((By.XPATH, xpath))
            )

            # Extraire le contenu spécifié pour chaque élément
            if content_type == 'text':
                result = [element.text for element in elements]
            elif content_type == 'href':
                result = [element.get_attribute('href') for element in elements]
            else:
                raise ValueError("Type de contenu non pris en charge. Utilisez 'text' ou 'href'.")

            return result

        except TimeoutException:
            print(f"Timeout de {timeout} secondes atteint. Réessai {retry + 1}/{max_retries}.")

        except NoSuchElementException:
            print(f"Élément non trouvé. Vérifiez votre XPath. Réessai {retry + 1}/{max_retries}.")

        except StaleElementReferenceException:
            print(f"Stale Element Reference Exception. L'élément n'est plus attaché au DOM. Réessai {retry + 1}/{max_retries}.")

        except Exception as e:
            print(f"Une erreur s'est produite : {str(e)}")

    print(f"Impossible de trouver et extraire le contenu après {max_retries} tentatives.")
    return []


In [4]:
def action_aleatoire(driver):
    actions_possibles = ["scroll_full", "scroll_half", "move_to_element"]
    action_choisie = random.choice(actions_possibles)

    # Enregistrer la position actuelle de la page
    current_scroll_position = driver.execute_script("return window.pageYOffset;")

    if action_choisie == "scroll_full":
        # Action aléatoire : Faites défiler la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    elif action_choisie == "scroll_half":
        # Action aléatoire : Faites défiler la moitié de la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
    elif action_choisie == "move_to_element":
        # Action aléatoire : Bougez la souris vers un élément aléatoire (par exemple, div de class PriceInformation_classifiedPrice__b-Jae)
        wait = WebDriverWait(driver, 10)
        element_to_move = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'center')))
        driver.execute_script("arguments[0].scrollIntoView();", element_to_move)

    # Revenir à la position enregistrée
    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

In [5]:
def initialize_driver():
    ua = UserAgent()
    user_agent = ua.random

    options = Options()
    options.add_argument('--headless')
    options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Firefox(options=options)
    return driver

## lien_ xpath utilisé

In [6]:
'''
V1
    
Prix moyen marché : dans la div de class o-YCHtV o-fHmpzP o-AxjCR o-bIMsfE o-fhuTtI
le p de class o-bkmzIL o-eZTujG o-eqqVmt o-bdcqQE
prix neuf : 
    div de class o-YCHtV o-fHmpzP o-AxjCR  
    le p de class o-bkmzIL o-eZTujG o-eqqVmt o-bdcqQE
'''

'\nV1\n    \nPrix moyen marché : dans la div de class o-YCHtV o-fHmpzP o-AxjCR o-bIMsfE o-fhuTtI\nle p de class o-bkmzIL o-eZTujG o-eqqVmt o-bdcqQE\nprix neuf : \n    div de class o-YCHtV o-fHmpzP o-AxjCR  \n    le p de class o-bkmzIL o-eZTujG o-eqqVmt o-bdcqQE\n'

# Initialize the driver
driver = initialize_driver()

# Load the page
driver.get("https://www.carwale.com/used/cars-for-sale/")

#  popup
popup_button_xpath = ".//span[contains(@class, 'o-frwuxB o-brXWGL o-cjsCNH o-dAFptd o-cDoIpQ o-cpnuEd o-dsiSgT o-NBTwp o-fuiuOo o-eoatGj dbDC7C o-eNbQSA o-brXWGL o-fzoTtm o-fmSMPH o-fznJFh o-fzptUA o-emXQxE o-fzptYC o-evuuYC ')]"

# Check popupis present 
try:
    popup_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, popup_button_xpath))
    )

    # Click 
    popup_button.click()

    

except Exception as e:
    print(f"An error occurred: {str(e)}")

# Define the XPath for the elements
xpath = ".//div[@class='o-fznJzu o-fznJPk o-fzptZB o-fzptOP']//a "

try:
    # Wait for the elements to be present
    elements_present = EC.presence_of_all_elements_located((By.XPATH, xpath))
    WebDriverWait(driver, 5).until(elements_present)

    # Call the scrape_content function to retrieve hrefs
    all_pages_href = scrape_content(driver, xpath, content_type='href')
    print(all_pages_href)
    try:
        for car_page in all_pages_href:
            # Open the link
            driver.get(car_page)
            print(car_page)
            # XPath for Marque + modele
            marque_modele_xpath = ".//h1[@class='o-dAvrcB o-eZTujG o-eqqVmt']"
            marque_modele = scrape_content(driver, marque_modele_xpath, content_type='text')
            print(marque_modele)
            # XPath for prix
            prix_xpath = ".//div[@class='o-cpnuEd o-dGBYL o-SoIQT o-Hyyko o-fzpilz o-fznJzb o-lIIwF o-fzptOP']//span[@class='o-eZTujG o-eqqVmt o-cscLpt']"
            prix = scrape_content(driver, prix_xpath, content_type='text')
            carac_xpath = ".//div[@class='lwnY3h l9NkHx o-YCHtV o-djSZRV o-cDkpxz o-bIMsfE o-dgboEW o-eZrAYv o-fbEicu o-bCRRBE']"
            carac_content = scrape_content(driver, carac_xpath, content_type='text')
            print("carc:", carac_content)
            print(prix)
            
            # spef
            
            span_spef_xpath = ".//div[contains(@class, 'o-brXWGL o-bqHweY o-OisZk o-frwuxB ME7bfG   o-bfyaNx')]//span[1]"
            click_success_spef = click_button(driver, span_spef_xpath)

            if click_success_spef:
                # Scroll jusqu'en bas de la page pour garantir que toutes les divs sont chargées
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)  # Ajouter une pause si nécessaire

                # Cliquer sur toutes les divs pour afficher le contenu
                divs_spef_xpath = ".//div[contains(@class, 'o-bIMsfE o-djSZRV o-GFmfi')]"
                #divs_spef_xpath = ".//div[contains(@class, 'o-cpnuEd o-frwuxB o-OisZk o-dsiSgT o-fzptZB o-fbZjxZ o-fzptOP o-fraaKH o-fzptVd o-fzptYr')]"
                divs_spef = driver.find_elements(By.XPATH, divs_spef_xpath)

                print("Nombre de divs_spef :", len(divs_spef))

                for div_spef in divs_spef:
                    try:
                        # Scroller chaque div dans la vue
                        driver.execute_script("arguments[0].scrollIntoView(true);", div_spef)
                        time.sleep(1)  # Ajouter une pause si nécessaire
                        div_spef.click()
                        time.sleep(2)  # Ajouter une pause si nécessaire
                    except:
                        # Ignorer les erreurs liées aux clics impossibles
                        pass

                # Récupérer le texte des li pour spef
                time.sleep(5)
                spef_ul_xpath = ".//ul[contains(@class, 'o-cpnuEd o-XylGE o-eNbQSA o-bIMsfE o-djSZRV o-GFmfi')]"
                spef_li_content = scrape_content(driver, spef_ul_xpath, content_type='text')

                # Utilisez spef_li_content pour les données récupérées
                print("Contenu pour spef :", spef_li_content)
            else:
                print("Erreur lors du clic sur spef.")

            # features
            
            # Click on the element with data-index="1"
            element_with_data_index_xpath = "./html/body/div[1]/div[2]/div[2]/div[1]/div/div[4]/div/div/div/div[1]/div/ul/li[2]/div/span"
            click_success_element_with_data_index = click_button(driver, element_with_data_index_xpath)

            if click_success_element_with_data_index:
                # Wait for some time to ensure the content is loaded after clicking
                time.sleep(2)

                # Now, you can perform actions on the element after clicking
                # Scroll jusqu'en bas de la page pour garantir que toutes les divs sont chargées
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)  # Ajouter une pause si nécessaire

                # Cliquer sur toutes les divs pour afficher le contenu
                divs_features_xpath = ".//div[contains(@class, 'o-bIMsfE o-djSZRV o-GFmfi')]"

                # divs_features_xpath = ".//div[contains(@class, 'o-cpnuEd o-frwuxB o-OisZk o-dsiSgT o-fzptZB o-fbZjxZ o-fzptOP o-fraaKH o-fzptVd o-fzptYr')]"
                divs_features = driver.find_elements(By.XPATH, divs_features_xpath)

                print("Nombre de divs_features :", len(divs_features))

                for div_features in divs_features:
                    try:
                        # Scroller chaque div dans la vue
                        driver.execute_script("arguments[0].scrollIntoView(true);", div_features)
                       
                        div_to_open_xpath = ".//div[contains(@class, 'o-cpnuEd o-frwuxB o-OisZk o-dsiSgT o-fzptZB o-fbZjxZ o-fzptOP o-fraaKH o-fzptVd o-fzptYr')]"
                        divs_to_open = driver.find_elements(By.XPATH,  div_to_open_xpath)
                       

                        # Wait for the element to become clickable
                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,  div_to_open_xpath)))

                        divs_to_open.click()
                        time.sleep(2)  # Ajouter une pause si nécessaire
                    except Exception as e:
                        # Ignorer les erreurs liées aux clics impossibles
                        print(f"Erreur lors du clic sur une div : {str(e)}")
                        pass

                # Récupérer le texte des li pour features
                features_ul_xpath = ".//ul[contains(@class, 'arv_ME o-cpnuEd o-XylGE o-eNbQSA o-bIMsfE o-djSZRV o-GFmfi')]"
                features_li_content = scrape_content(driver, features_ul_xpath, content_type='text')

                # Utilisez features_li_content pour les données récupérées
                print("Contenu pour features :", features_li_content)
            else:
                print(f"Erreur lors du clic sur l'élément avec data-index='1'.")






    except Exception as e:
        print(f"An error occurred: {str(e)}")
except TimeoutException:
    print("Timed out waiting for page to load")

finally:
    # Close the driver when done
    driver.quit()


## v2

In [ ]:
# Initialize the driver
driver = initialize_driver()

# Load the page
driver.get("https://www.carwale.com/used/cars-for-sale/")

#  popup
popup_button_xpath = ".//span[contains(@class, 'o-frwuxB o-brXWGL o-cjsCNH o-dAFptd o-cDoIpQ o-cpnuEd o-dsiSgT o-NBTwp o-fuiuOo o-eoatGj dbDC7C o-eNbQSA o-brXWGL o-fzoTtm o-fmSMPH o-fznJFh o-fzptUA o-emXQxE o-fzptYC o-evuuYC ')]"

# Check popupis present 
try:
    popup_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, popup_button_xpath))
    )

    # Click 
    popup_button.click()

    

except Exception as e:
    print(f"An error occurred: {str(e)}")

# Define the XPath for the elements
xpath = ".//div[@class='o-fznJzu o-fznJPk o-fzptZB o-fzptOP']//a "

try:
    # Wait for the elements to be present
    elements_present = EC.presence_of_all_elements_located((By.XPATH, xpath))
    WebDriverWait(driver, 5).until(elements_present)

    # Call the scrape_content function to retrieve hrefs
    all_pages_href = scrape_content(driver, xpath, content_type='href')
    print(all_pages_href)
    try:
        for car_page in all_pages_href:
            # Open the link
            driver.get(car_page)
            print(car_page)
            # XPath for Marque + modele
            marque_modele_xpath = ".//h1[@class='o-dAvrcB o-eZTujG o-eqqVmt']"
            marque_modele = scrape_content(driver, marque_modele_xpath, content_type='text')
            print(marque_modele)
            # XPath for prix
            prix_xpath = ".//div[@class='o-cpnuEd o-dGBYL o-SoIQT o-Hyyko o-fzpilz o-fznJzb o-lIIwF o-fzptOP']//span[@class='o-eZTujG o-eqqVmt o-cscLpt']"
            prix = scrape_content(driver, prix_xpath, content_type='text')
            carac_xpath = ".//div[@class='lwnY3h l9NkHx o-YCHtV o-djSZRV o-cDkpxz o-bIMsfE o-dgboEW o-eZrAYv o-fbEicu o-bCRRBE']"
            carac_content = scrape_content(driver, carac_xpath, content_type='text')
            print("carc:", carac_content)
            print(prix)
            
            # spef
            
            span_spef_xpath = ".//div[contains(@class, 'o-brXWGL o-bqHweY o-OisZk o-frwuxB ME7bfG   o-bfyaNx')]//span[1]"
            click_success_spef = click_button(driver, span_spef_xpath)

            if click_success_spef:
                # Scroll jusqu'en bas de la page pour garantir que toutes les divs sont chargées
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)  # Ajouter une pause si nécessaire

                # Cliquer sur toutes les divs pour afficher le contenu
                divs_spef_xpath = ".//div[contains(@class, 'o-bIMsfE o-djSZRV o-GFmfi')]"
                #divs_spef_xpath = ".//div[contains(@class, 'o-cpnuEd o-frwuxB o-OisZk o-dsiSgT o-fzptZB o-fbZjxZ o-fzptOP o-fraaKH o-fzptVd o-fzptYr')]"
                divs_spef = driver.find_elements(By.XPATH, divs_spef_xpath)

                print("Nombre de divs_spef :", len(divs_spef))
                div_to_open_xpath = ".//div[contains(@class, 'o-cpnuEd o-frwuxB o-OisZk o-dsiSgT o-fzptZB o-fbZjxZ o-fzptOP o-fraaKH o-fzptVd o-fzptYr')]"
                divs_to_open = driver.find_elements(By.XPATH, div_to_open_xpath)

                for div_spef in divs_spef:
                    try:
                        # Scroller chaque div dans la vue
                        driver.execute_script("arguments[0].scrollIntoView(true);", div_spef)
                        time.sleep(1)  # Ajouter une pause si nécessaire
                        div_spef.click()
                        time.sleep(2)  # Ajouter une pause si nécessaire
                    except:
                        # Ignorer les erreurs liées aux clics impossibles
                        pass

                # Récupérer le texte des li pour spef
                time.sleep(5)
                spef_ul_xpath = ".//ul[contains(@class, 'o-cpnuEd o-XylGE o-eNbQSA o-bIMsfE o-djSZRV o-GFmfi')]"
                spef_li_content = scrape_content(driver, spef_ul_xpath, content_type='text')

                # Utilisez spef_li_content pour les données récupérées
                print("Contenu pour spef :", spef_li_content)
            else:
                print("Erreur lors du clic sur spef.")

            # features
            
            # Click on the element with data-index="1"
            element_with_data_index_xpath = "./html/body/div[1]/div[2]/div[2]/div[1]/div/div[4]/div/div/div/div[1]/div/ul/li[2]/div/span"
            click_success_element_with_data_index = click_button(driver, element_with_data_index_xpath)

            if click_success_element_with_data_index:
                # Wait for some time to ensure the content is loaded after clicking
                time.sleep(2)

                # Now, you can perform actions on the element after clicking
                # Scroll jusqu'en bas de la page pour garantir que toutes les divs sont chargées
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)  # Ajouter une pause si nécessaire

                # Cliquer sur toutes les divs pour afficher le contenu
               
                div_to_open_xpath = ".//div[contains(@class, 'o-cpnuEd o-frwuxB o-OisZk o-dsiSgT o-fzptZB o-fbZjxZ o-fzptOP o-fraaKH o-fzptVd o-fzptYr')]"
                divs_to_open = driver.find_elements(By.XPATH, div_to_open_xpath)

                print("Nombre de divs_to_open :", len(divs_to_open))

                # Initialize an empty set to store unique features
                unique_features = set()

                for div_to_open in divs_to_open:
                    try:
                        # Scroller chaque div dans la vue
                        driver.execute_script("arguments[0].scrollIntoView(true);", div_to_open)

                        # Click on the div using JavaScript
                        driver.execute_script("arguments[0].click();", div_to_open)
                        time.sleep(2)  # Ajouter une pause si nécessaire

                        # Récupérer le texte des li pour features
                        features_ul_xpath = ".//ul[contains(@class, 'arv_ME o-cpnuEd o-XylGE o-eNbQSA o-bIMsfE o-djSZRV o-GFmfi')]"
                        features_li_content = scrape_content(driver, features_ul_xpath, content_type='text')

                        # Convert the list to a tuple before adding to the set
                        features_tuple = tuple(features_li_content)
                        unique_features.add(features_tuple)
                    except Exception as e:
                        # Ignorer les erreurs liées aux clics impossibles
                        print(f"Erreur lors du clic sur une div : {str(e)}")
                        pass

                # Convert the set of tuples to a list if needed
                unique_features_list = [list(item) for item in unique_features]

                # Print the list containing unique features
                print("Contenu pour les features uniques :", unique_features_list)

               







    except Exception as e:
        print(f"An error occurred: {str(e)}")
except TimeoutException:
    print("Timed out waiting for page to load")

finally:
    # Close the driver when done
    driver.quit()


An error occurred: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

['https://www.carwale.com/used/cars-in-gurgaon/bmw-6-series-gt/d4228881/?slot=0&dc=0&rk=1', 'https://www.carwale.com/used/cars-in-mumbai/maruti-suzuki-ciaz/d4210377/?slot=0&dc=0&rk=2', 'https://www.carwale.com/used/cars-in-delhi/bmw-6-series-gt/d4245897/?slot=0&dc=0&rk=3', 'https://www.carwale.com/used/cars-in-delhi/mercedes-benz-e-class/d4272029/?slot=0&dc=0&rk=4', 'https://www.carwale.com/used/cars-in-mumbai/bmw-3-series/d4304595/?slot=0&dc=0&rk=5', 'https://www.carwale.com/used/cars-in-mumbai/volvo-v40-cross-country/d4322515/?slot=0&dc=0&rk=6', 'https://www.carwale.com/used/cars-in-hyderabad/volkswagen-vento/d4224265/?slot=0&dc=0&rk=7', 'https://www.carwale.c